In [1]:
import numpy as np
import pandas as pd
from keras.utils import np_utils
from keras import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


### 本地csv中导入mnist数据 

In [3]:
train = pd.read_csv("./CNN for MINST dataset/train.csv")
test = pd.read_csv("./CNN for MINST dataset/test.csv")
target=train['label']
features=train.drop(labels='label',axis=1)#pd.drop():Return new object with labels in requested axis removed.
#print(target.shape,features.shape)
## normalize
features=features.astype('float32')
test=test.astype('float32')
features/=255.0
test/=255.0
## Reshape image in 3 dimensions (height = 28, width = 28 , channel = 1)
height,width ,channel= 28,28,1
features=features.values.reshape(-1,height,width,channel)
test=test.values.reshape(-1,height,width,channel)

## Encode labels to one hot vectors
target=to_categorical(target,num_classes=10)

## Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(features, target, test_size = 0.1)

### keras 库中导入mnist数据

In [3]:
from keras.datasets import mnist
(trainx,trainy),(testx,testy)=mnist.load_data()
### convert mnist data to 28x28x1
X_train=trainx.reshape([-1,28,28,1])
x_test=testx.reshape([testx.shape[0],28,28,1])
### normlized
X_train=X_train.astype('float32')
x_test=x_test.astype('float32')
### 将label 值转换为需要的格式(one-hot编码)
Y_train=to_categorical(trainy,10)
y_test=to_categorical(testy,10)
print(X_train.shape,x_test.shape,Y_train.shape,type(trainx))
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1)

(60000, 28, 28, 1) (10000, 28, 28, 1) (60000, 10) <class 'numpy.ndarray'>


### 构建 LeNet-5 model

In [4]:
model = Sequential()
# 卷积层
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (28,28,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
# 全连接层
model.add(Flatten())#Flatten():将输入展平。不影响批量大小
model.add(Dense(512, activation = "relu", use_bias= True))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax", use_bias= True))

### fit results

In [ ]:
model.compile(optimizer = 'adam' , loss = "categorical_crossentropy", metrics=["accuracy"])
## 使用 validation_split 分割数据
#results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=50)
## 使用train_test_split()分割的validation_data
fit_results = model.fit(X_train, Y_train, validation_data = (X_val,Y_val), batch_size=16, epochs=50)

### predict results 为输入样本test生成输出预测

In [ ]:
pre_results = model.predict(test)
# select the indix with the maximum probability
pre_results = np.argmax(results,axis = 1)
pre_results = pd.Series(pre_results,name="Label")
# 预测结果save到本地csv文件
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),pre_results],axis = 1)
submission.to_csv("./CNN for MINST dataset/LeNet_test_prediction.csv",index=False)

In [ ]:
#Plot Loss and accuracy
import matplotlib.pyplot as plt
plt.figure(figsize = (15,5))
plt.subplot(1,2,1)
plt.plot(fit_results.history['acc'])
plt.plot(fit_results.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(1,2,2)
plt.plot(fit_results.history['loss'])
plt.plot(fit_results.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.show()